In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [5]:
start_date = '2022-12-19'
end_date = '2022-12-25'

In [15]:
validate = f"""

with fe as
(
select
         yyyymmdd,
         substr(profile_phone, -10, 10) as mobile,
         trim(eventprops_currentcity) city,
         eventprops_epoch,eventProps_userId,
         eventprops_ctsessionid,
         substr(profile_phone, -10, 10) ||'-'|| cast(eventprops_ctsessionid as varchar) unique_id
         from
            hive.raw.clevertap_customer_fareestimate
         where
            yyyymmdd >= date_format(cast('{start_date}' as date), '%Y%m%d') 
            AND yyyymmdd <= date_format(cast('{end_date}' as date), '%Y%m%d')
            AND eventprops_currentcity != '' 
            AND eventprops_currentcity != 'NA'
            AND eventprops_currentcity IN ('Chennai', 'Delhi', 'Hyderabad')


),
rr as
(
select         
         yyyymmdd,
         substr(profile_phone, -10, 10) as mobile,
         trim(eventprops_currentcity) city,
         eventprops_epoch,eventProps_userId,
         eventprops_ctsessionid,
         substr(profile_phone, -10, 10) ||'-'|| cast(eventprops_ctsessionid as varchar) unique_id
         from
            hive.raw.clevertap_customer_request_rapido
         where
            yyyymmdd >= date_format(cast('{start_date}' as date), '%Y%m%d') 
            AND yyyymmdd <= date_format(cast('{end_date}' as date), '%Y%m%d')
            AND eventprops_currentcity != '' 
            AND eventprops_currentcity != 'NA'
            AND eventprops_currentcity IN ('Chennai', 'Delhi', 'Hyderabad')

)

select
--yyyymmdd,
city,
'fe_data' as source,
count(distinct unique_id) session_count
-- count(eventprops_ctsessionid) as session_count
from fe
group by 1,2


union

select
--yyyymmdd,
city,
'rr_data' as source,
count(distinct unique_id) session_count
-- count(eventprops_ctsessionid) as session_count
from rr
group by 1,2


order by 1,2

"""

In [16]:
df_query = pd.read_sql(validate, conn)
df_query

,city,source,session_count
0,Chennai,fe_data,1149965
1,Chennai,rr_data,716487
2,Delhi,fe_data,1767140
3,Delhi,rr_data,995555
4,Hyderabad,fe_data,3268880
5,Hyderabad,rr_data,1860015
